In [ ]:
import torch 
import torch.nn as nn
import torch.utils.data as td

torch.manual_seed(0)
print("Libraries imported - ready to use PyTorch", torch.__version__)

In [ ]:
tensor_x = torch.Tensor(train_x_vals).float()
tensor_y = torch.Tensor(train_y_vals).float()
train_ds = td.TensorDataset(tensor_x,tensor_y)
train_loader = td.DataLoader(train_ds,batch_size=20,shuffle=True)
print(train_loader)

tensor_x_test = torch.Tensor(test_x_vals).float()
tensor_y_test = torch.Tensor(test_y_vals).float()
test_ds = td.TensorDataset(tensor_x_test,tensor_y_test)
test_loader = td.DataLoader(test_ds,shuffle=False,batch_size=20)
print(test_loader)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        self.fc1 = nn.Linear(train_x_vals.shape[1],10)
        self.fc2 = nn.Linear(10,20)
        self.fc3 = nn.Linear(20,10)
        self.fc4 = nn.Linear(10,train_y.shape[1])
        
    def forward(self,X):
        out = torch.relu(self.fc1(X))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = self.fc4(out)
        return out

In [ ]:
def train(model,data_loader,optimizer,criterion):
    model.train()
    train_loss = 0
    
    for batch,tensor in enumerate(data_loader):
        data,target = tensor
        
        ## reset optimizer
        optimizer.zero_grad()
        
        ## forward propagation
        out = model(data)
#         print(data,target,out)
#         print('-------------------------')
        loss = criterion(out,target)
        train_loss += loss.item()
#         print(batch,train_loss)
        ## back propagation
        loss.backward()
        optimizer.step()
        
    avg_loss = train_loss/(batch+1)
    return avg_loss

def test(model,data_loader,criterion):
    model.eval()
    test_loss = 0
    correct = 0
    
    for batch,tensor in enumerate(data_loader):
        data,target = tensor
        out = model(data)
        loss = criterion(out,target)
        test_loss += loss.item()
        
        predicted = out > 0.5
        correct += torch.sum(predicted == target).item()
    acc = correct/len(data_loader.dataset)
    avg_loss = test_loss/(batch+1)
    return avg_loss,acc

In [ ]:
N_EPOCHS = 100
criterion = nn.BCEWithLogitsLoss()
learn_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(),learn_rate)
model = NeuralNet()
print(model)


train_losses = []
test_losses = []
test_acces = []

for epoch in range(N_EPOCHS):
    tr_loss = train(model,train_loader,optimizer,criterion)
    te_loss,te_acc = test(model,test_loader,criterion)
    train_losses.append(tr_loss)
    test_losses.append(te_loss)
    test_acces.append(te_acc)
    if epoch % 10 == 0: 
        print(f'Epoch : {epoch},train loss : {tr_loss},test acc : {te_acc},test loss : {te_loss}')